## 기본 셋팅

In [115]:
import math
import collections

import numpy as np
import sympy as sy
import matplotlib.pyplot as plt

from scipy.optimize import fsolve

from coppeliasim_zmqremoteapi_client import RemoteAPIClient

In [116]:
# Connect to the simulator
client = RemoteAPIClient()
sim = client.require("sim")

print("Connected to CoppeliaSim")

Connected to CoppeliaSim


In [117]:
# Get arm's joints
joints = []
for i in range(5):
    joints.append(sim.getObject(f"/youBotArmJoint{i}"))

# Gripper joint
joints.append(sim.getObject(f"/youBotGripperJoint2"))

points = []

# Car reference frame
points.append(sim.getObject(f"/youBot_ref"))
# Joint-0 위치
points.append(sim.getObject(f"/p0_ref"))
# End Effector 위치
points.append(sim.getObject(f"/pe_ref"))
# Target 위치
points.append(sim.getObject(f"/Target"))

# joint 제어 모드 변경
for joint in joints:
    sim.setObjectInt32Param(
        joint,
        sim.jointintparam_dynctrlmode,
        sim.jointdynctrl_position,
    )

In [118]:
# joint angle 조회
def read_joints(joints):
    js = []
    for joint in joints:
        j = sim.getJointPosition(joint)
        js.append(j)
    return js

print("Initial joint angles:", read_joints(joints))

Initial joint angles: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


In [119]:
# point position & orientation 조회
def read_points(points):
    ps = []
    for point in points:
        p = sim.getObjectPosition(point)
        o = sim.getObjectOrientation(point)
        ps.append(np.array(p + o))
    return ps

def print_points(ps):
    for i, p in enumerate(ps):
        print(f"Point {i+1}: Position = {p[:3]}, Orientation = {p[3:]}")

ps = read_points(points)
print("Initial point positions and orientations:")
print_points(ps)

Initial point positions and orientations:
Point 1: Position = [2.50000000e-02 9.99960000e-01 1.53971835e-11], Orientation = [ 3.71924713e-15 -1.44328993e-15 -1.66533454e-16]
Point 2: Position = [0.0249  1.16618 0.099  ], Orientation = [-5.55111512e-17 -1.11022302e-16  5.55111512e-17]
Point 3: Position = [0.0249  1.19918 0.74   ], Orientation = [-2.52598440e-16 -4.22626393e-16  4.79537226e-17]
Point 4: Position = [0.075 1.6   0.02 ], Orientation = [-0.  0. -0.]


In [120]:
# joint 제어
def control_joint(joints, thetas):
    for joint, j in zip(joints, thetas):
        sim.setJointTargetPosition(joint, j)

In [121]:
# gripper 제어
def control_gripper(gripper, state):
    position = sim.getJointPosition(gripper)
    position += 0.005 if state else -0.005
    sim.setJointTargetPosition(gripper, position)
    return position

In [122]:
# control joint velocity

def trace_arm(joints, target_thetas):
    js = read_joints(joints)
    diff_sum = 0
    thetas = []
    for i, target in enumerate(target_thetas):
        diff = js[i] - target
        diff_sum += diff
        thetas.append(js[i] - min(0.1, max(-0.05, diff)))
    control_joint(joints, thetas)
    return diff_sum

In [123]:
# forward kinematics
def fk(thetas, params):
    j1, j2, j3, j4 = thetas[:4]
    j0, pc = params[:2]
    # 월드 기준 자동차
    dc = pc[5]
    TWC = np.array([
        [np.cos(dc), -np.sin(dc), 0, pc[0]],
        [np.sin(dc),  np.cos(dc), 0, pc[1]],
        [         0,           0, 1, pc[2]],
        [         0,           0, 0,     1]
    ])

    # 자동차 -> joint-0
    TC0 = np.array([ # 좌표이동 및 y축을 기준으로 90도 회전
        [1, 0, 0, 0.0],
        [0, 1, 0, 0.166],
        [0, 0, 1, 0.099],
        [0, 0, 0, 1]
    ]) @ np.array([
        [np.cos(j0), -np.sin(j0), 0, 0],
        [np.sin(j0),  np.cos(j0), 0, 0],
        [         0,           0, 1, 0],
        [         0,           0, 0, 1]
    ])
    TW0 = TWC @ TC0

    # joint-0 -> joint-1
    ay1 = np.pi / 2
    T01 = np.array([ # 좌표이동 및 y축을 기준으로 90도 회전
        [ np.cos(ay1), 0, np.sin(ay1), 0.0],
        [           0, 1,           0, 0.033],
        [-np.sin(ay1), 0, np.cos(ay1), 0.147],
        [           0, 0,           0, 1]
    ]) @ np.array([ # z축을 기준으로 j1만큼 회전
        [np.cos(j1), -np.sin(j1), 0, 0],
        [np.sin(j1),  np.cos(j1), 0, 0],
        [         0,           0, 1, 0],
        [         0,           0, 0, 1]
    ])
    TW1 = TW0 @ T01

    # joint-1 -> joint-2
    T12 = np.array([ # 좌표이동, 회전 없음
        [1, 0, 0, -0.155],
        [0, 1, 0,  0.0],
        [0, 0, 1,  0.0],
        [0, 0, 0,  1]
    ]) @ np.array([ # z축을 기준으로 j2만큼 회전
        [np.cos(j2), -np.sin(j2), 0, 0],
        [np.sin(j2),  np.cos(j2), 0, 0],
        [         0,           0, 1, 0],
        [         0,           0, 0, 1]
    ])
    TW2 = TW1 @ T12

    # joint-2 -> joint-3
    T23 = np.array([ # 좌표이동, 회전 없음
        [1, 0, 0, -0.135],
        [0, 1, 0,  0.0],
        [0, 0, 1,  0.0],
        [0, 0, 0,  1]
    ]) @ np.array([ # z축을 기준으로 j3만큼 회전
        [np.cos(j3), -np.sin(j3), 0, 0],
        [np.sin(j3),  np.cos(j3), 0, 0],
        [         0,           0, 1, 0],
        [         0,           0, 0, 1]
    ])
    TW3 = TW2 @ T23

    # joint-3 -> joint-4
    ay4 = -np.pi / 2
    T34 = np.array([ # 좌표이동 및 y축을 기준으로 -90도 회전
        [ np.cos(ay4), 0, np.sin(ay4), -0.081],
        [           0, 1,           0,  0.0],
        [-np.sin(ay4), 0, np.cos(ay4),  0.0],
        [           0,  0,          0,  1]
    ]) @ np.array([ # z축을 기준으로 j4만큼 회전
        [np.cos(j4), -np.sin(j4), 0, 0],
        [np.sin(j4),  np.cos(j4), 0, 0],
        [         0,           0, 1, 0],
        [         0,           0, 0, 1]
    ])
    TW4 = TW3 @ T34

    pe_hat = TW4 @ np.array([ 0.0,   0.0,   0.123, 1])

    return pe_hat[:3]

In [124]:
# inverse kinematics
def ik(thetas, params):  
    pt = params[-1][:3]
    pe_hat = fk(thetas, params)
    # theta 범위 검증
    if thetas[0] < np.deg2rad(-90) or np.deg2rad(75) < thetas[0]:
        return 10, 0, 0, 0
    elif thetas[1] < np.deg2rad(-131.00) or np.deg2rad(131.00) < thetas[1]:
        return 10, 0, 0, 0
    elif thetas[2] < np.deg2rad(-102.00) or np.deg2rad(102.00) < thetas[2]:
        return 10, 0, 0, 0
    elif thetas[3] < np.deg2rad(-90.00) or np.deg2rad(90.00) < thetas[3]:
        return 10, 0, 0, 0
    return np.linalg.norm(pe_hat - pt), 0, 0, 0

In [125]:
def solve(js, ps):
    p0, pt = ps[1], ps[-1]
    diff = pt[:2] - p0[:2]
    angle = math.atan2(diff[1], diff[0])
    j0 = angle - p0[-1] - np.pi / 2
    target_thetas = fsolve(
        ik,
        [js[1], js[2], js[3], js[4]],
        [j0, ps[0], ps[-1]]
    )
    return np.concatenate((np.array([j0]), target_thetas))

In [126]:
n_steps = 400

# 시뮬레이션 실행
sim.setStepping(True)
sim.startSimulation()

# 각 스텝 실행
js = read_joints(joints)
ps = read_points(points)
target_thetas = solve(js, ps)
place_thetas = [np.pi, -np.pi / 6, -np.pi / 2.7, -np.pi / 3, 0]
base_thetas = [0, 0, 0, 0, 0]
stage = 0
gripper_positions = collections.deque(maxlen=100)
for i in range(n_steps):
    if stage == 0: # move to target
        diff_sum = trace_arm(joints, target_thetas)
        if abs(diff_sum) < 0.005:
            stage = 1
    elif stage == 1: # grip target
        position = control_gripper(joints[-1], True)
        gripper_positions.append(position)
        if len(gripper_positions) > 5 and np.abs(gripper_positions[-1] - gripper_positions[-5]) < 0.001:
            stage = 2
    elif stage == 2:  # pick the target
        target_thetas[0] = np.pi
        target_thetas[1] = -np.pi / 12
        target_thetas[2] = -np.pi / 6
        diff_sum = trace_arm(joints, target_thetas)
        if abs(diff_sum) < 0.005:
            stage = 3
    elif stage == 3: # move to place
        diff_sum = trace_arm(joints, place_thetas)
        if abs(diff_sum) < 0.005:
            stage = 4
    elif stage == 4: # place the target
        position = control_gripper(joints[-1], False)
        gripper_positions.append(position)
        if len(gripper_positions) > 5 and np.abs(gripper_positions[-1] - gripper_positions[-5]) < 0.001:
            stage = 5
    elif stage == 5: # move to base
        diff_sum = trace_arm(joints, base_thetas)
        if abs(diff_sum) < 0.005:
            stage = 6
    sim.step()

# 시뮬레이션 종료
sim.stopSimulation()

C:\Users\joy4m\AppData\Local\Temp\ipykernel_154252\3481707928.py:6: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last five Jacobian evaluations.
  target_thetas = fsolve(


# Initialize 

In [127]:
for i in range(len(joints)):
    sim.setJointPosition(joints[i], 0)

# Collision Check & visualize

In [128]:
def check_collision(sim, config, obstacle_handle):
    """Check if a configuration collides with obstacles"""
    # Get car position and orientation for base transform
    car_handle = sim.getObject("/youBot_ref")
    car_position = sim.getObjectPosition(car_handle, -1)
    car_orientation = sim.getObjectOrientation(car_handle, -1)
    base_params = [config[0], [*car_position, *car_orientation], None]
    
    # Get handles for all robot parts
    robot_parts = [
        sim.getObject("/Rectangle22"),
        sim.getObject("/Rectangle10"),
        sim.getObject("/Rectangle9"),
        sim.getObject("/Rectangle8"),
        sim.getObject("/Rectangle7"),
        sim.getObject("/Rectangle6"),
        sim.getObject("/Rectangle5")
    ]
    
    # Update robot configuration
    for i, joint in enumerate(joints[:len(config)]):
        sim.setJointPosition(joint, config[i])
    
    # Check collision between each robot part and the obstacle
    for part in robot_parts:
        isColliding = check_distance_to_sphere(
            sim, 
            part,
            obstacle_handle,
            0.2  # sphere radius
        )
        if isColliding:
            return True
            
    return False

def check_distance_to_sphere(sim, part_handle, obstacle_handle, sphere_radius):
    """Check if a robot part collides with a spherical obstacle"""
    # Get positions
    part_pos = sim.getObjectPosition(part_handle, -1)
    obstacle_pos = sim.getObjectPosition(obstacle_handle, -1)
    
    # Get minimal distance between part and obstacle center
    dx = part_pos[0] - obstacle_pos[0]
    dy = part_pos[1] - obstacle_pos[1]
    dz = part_pos[2] - obstacle_pos[2]
    distance = np.sqrt(dx*dx + dy*dy + dz*dz)
    
    # Simple sphere collision check
    # Note: This is a simplified check. For more accurate results,
    # you might want to consider the actual shape of the robot parts
    return distance < sphere_radius

In [129]:
def create_colored_dummy(sim, position, name, color):
    """Create a colored dummy object in CoppeliaSim"""
    if isinstance(position, np.ndarray):
        position = position.tolist()
        
    dummy_handle = sim.createDummy(0.05)  # Size of dummy
    sim.setObjectPosition(dummy_handle, -1, position)
    sim.setObjectAlias(dummy_handle, f"Sample_{name}")
    
    # Set dummy color
    try:
        sim.setShapeColor(dummy_handle, None, 0, color)
    except:
        pass
    return dummy_handle

def create_dummy(sim, position, name):
    """Create a default dummy object in CoppeliaSim"""
    return create_colored_dummy(sim, position, name, [0, 1, 0])  # Default green color

def visualize_colored_edge(sim, start_pos, end_pos, color):
    """Draw a colored edge between two configurations"""
    if isinstance(start_pos, np.ndarray):
        start_pos = start_pos.tolist()
    if isinstance(end_pos, np.ndarray):
        end_pos = end_pos.tolist()
        
    return sim.addDrawingObject(
        sim.drawing_lines, 2, 0, -1, 0, color,
        [start_pos[0], start_pos[1], start_pos[2],
         end_pos[0], end_pos[1], end_pos[2]]
    )

def visualize_edge(sim, start_pos, end_pos):
    """Draw a default edge between two configurations"""
    return visualize_colored_edge(sim, start_pos, end_pos, [0, 1, 0])  # Default green color

# RRT

In [130]:
class Node:
    """Tree node for RRT"""
    def __init__(self, config):
        # Always store config as numpy array
        self.config = np.array(config) if not isinstance(config, np.ndarray) else config
        self.parent = None
        self.children = []

class RRTPlanner:
    def __init__(self, sim, start_config, goal_config, epsilon=0.3):
        self.sim = sim
        # Convert configs to numpy arrays
        self.start_config = np.array(start_config)
        self.goal_config = np.array(goal_config)
        self.epsilon = epsilon
        self.nodes = []
        self.dummy_handles = []
        self.line_handles = []
        
        # Joint ranges
        self.joint_ranges = [
            (-np.pi, np.pi),          # Joint 0: -180 to 180 degrees
            (-np.pi/2, np.pi*75/180), # Joint 1: -90 to 75 degrees
            (-np.pi*131/180, np.pi*131/180), # Joint 2: -131 to 131 degrees
            (-np.pi*102/180, np.pi*102/180), # Joint 3: -102 to 102 degrees
            (-np.pi/2, np.pi/2)       # Joint 4: -90 to 90 degrees
        ]
        
        # Initialize tree with start configuration
        self.root = Node(self.start_config)
        self.nodes.append(self.root)
        
        try:
            # Visualize start point
            self._visualize_node(self.root, color=[1,0,0])  # Red for start
        except Exception as e:
            print(f"Visualization error: {e}")

    def sample_config(self):
        """Generate random configuration"""
        config = np.array([np.random.uniform(min_val, max_val) 
                         for min_val, max_val in self.joint_ranges])
        return config
    
    def find_nearest_node(self, q_rand):
        """Find nearest node in the tree"""
        # Ensure q_rand is numpy array
        q_rand = np.array(q_rand)
        
        distances = [np.linalg.norm(node.config - q_rand) for node in self.nodes]
        nearest_idx = np.argmin(distances)
        return self.nodes[nearest_idx]
    
    def new_config(self, q_near, q_rand):
        """Generate new configuration by moving from q_near toward q_rand"""
        q_rand = np.array(q_rand)
        direction = q_rand - q_near.config
        distance = np.linalg.norm(direction)
        
        if distance < self.epsilon:
            return q_rand
            
        # Normalize and scale by epsilon
        direction = direction / distance * self.epsilon
        return q_near.config + direction
    
    def _visualize_node(self, node, color=None):
        """Visualize a node and its connection to parent"""
        try:
            # Get end effector position for visualization
            car_handle = self.sim.getObject("/youBot_ref")
            car_position = self.sim.getObjectPosition(car_handle, -1)
            car_orientation = self.sim.getObjectOrientation(car_handle, -1)
            
            # Convert configurations to list for fk function
            node_config = node.config.tolist()
            base_params = [node_config[0], [*car_position, *car_orientation], None]
            ee_pos = fk(node_config, base_params)
            
            # Create dummy for node
            if color is None:
                color = [0, 1, 0]  # Green by default
            dummy_handle = create_colored_dummy(self.sim, ee_pos, f"node_{len(self.nodes)}", color)
            self.dummy_handles.append(dummy_handle)
            
            # Draw line to parent if exists
            if node.parent is not None:
                parent_config = node.parent.config.tolist()
                parent_ee_pos = fk(parent_config, base_params)
                line_handle = visualize_colored_edge(self.sim, parent_ee_pos, ee_pos, color)
                self.line_handles.append(line_handle)
                
            self.sim.step()
        except Exception as e:
            print(f"Visualization error: {e}")
    
    def extend(self):
        """Extend tree by one node"""
        try:
            # With some probability, sample toward goal
            if np.random.random() < 0.1:  # 10% chance to sample goal
                q_rand = self.goal_config
            else:
                q_rand = self.sample_config()
            
            # Find nearest node
            q_near = self.find_nearest_node(q_rand)
            
            # Generate new configuration
            q_new_config = self.new_config(q_near, q_rand)
            
            # Check if new configuration is in collision
            if check_collision(self.sim, q_new_config.tolist(), self.sim.getObject("/Obstacle")):
                return None
                
            # Create new node
            q_new = Node(q_new_config)
            q_new.parent = q_near
            q_near.children.append(q_new)
            self.nodes.append(q_new)
            
            # Visualize new node
            self._visualize_node(q_new)
            
            return q_new
        except Exception as e:
            print(f"Extension error: {e}")
            return None
    
    def plan(self, max_iterations=1000):
        """Plan path from start to goal"""
        print("Planning RRT path...")
        goal_threshold = 0.1  # Distance threshold to consider goal reached
        
        for i in range(max_iterations):
            # Extend tree
            new_node = self.extend()
            if new_node is None:
                continue
                
            # Check if goal is reached
            dist_to_goal = np.linalg.norm(new_node.config - self.goal_config)
            if dist_to_goal < goal_threshold:
                print(f"Goal reached after {i+1} iterations!")
                goal_node = Node(self.goal_config)
                self._visualize_node(goal_node, color=[0,0,1])  # Blue for goal
                return self.extract_path(new_node)
                
        print("Failed to reach goal after maximum iterations")
        return None
    
    def extract_path(self, goal_node):
        """Extract path from start to goal"""
        path = []
        current = goal_node
        
        while current is not None:
            path.append(current.config.tolist())  # Convert to list for compatibility
            current = current.parent
            
        return list(reversed(path))
    
    def cleanup_visualization(self):
        """Remove all visualization objects"""
        try:
            for handle in self.dummy_handles:
                self.sim.removeObject(handle)
            for handle in self.line_handles:
                self.sim.removeDrawingObject(handle)
        except Exception as e:
            print(f"Cleanup error: {e}")

def plan_with_rrt(sim, start_config, goal_config):
    """Plan path using RRT and prepare it for execution"""
    try:
        # Initialize RRT planner
        rrt = RRTPlanner(sim, start_config, goal_config, epsilon=0.1)
        
        # Plan path
        path = rrt.plan()
        
        if path is None:
            print("No path found!")
            rrt.cleanup_visualization()
            return None
        
        return path, rrt
        
    except Exception as e:
        print(f"Planning error: {e}")
        return None

# RRT_goal sample

In [108]:
class RRT_GOAL_Planner(RRTPlanner):
    def __init__(self, sim, start_config, goal_config, epsilon=0.2, goal_bias=0.05):
        super().__init__(sim, start_config, goal_config, epsilon)
        self.goal_bias = goal_bias  # Probability of sampling toward goal
        print(f"Initialized RRT Goal-Biased Planner with {goal_bias*100}% goal bias")
        
    def extend(self):
        """Extend tree by one node with 5% goal bias"""
        try:
            # With 5% probability, sample toward goal
            if np.random.random() < self.goal_bias:  # 5% chance to sample goal
                q_rand = self.goal_config
                # print("Sampling toward goal")  # Optional debug print
            else:
                q_rand = self.sample_config()
                # print("Uniform sampling")  # Optional debug print
            
            # Find nearest node
            q_near = self.find_nearest_node(q_rand)
            
            # Generate new configuration
            q_new_config = self.new_config(q_near, q_rand)
            
            # Check if new configuration is in collision
            if check_collision(self.sim, q_new_config.tolist(), self.sim.getObject("/Obstacle")):
                return None
                
            # Create new node
            q_new = Node(q_new_config)
            q_new.parent = q_near
            q_near.children.append(q_new)
            self.nodes.append(q_new)
            
            # Visualize new node
            self._visualize_node(q_new)
            
            return q_new
        except Exception as e:
            print(f"Extension error: {e}")
            return None

def plan_with_rrt_goal(sim, start_config, goal_config, goal_bias=0.05):
    """Plan path using RRT with goal bias"""
    try:
        # Initialize RRT planner with goal bias
        rrt = RRT_GOAL_Planner(sim, start_config, goal_config, epsilon=0.1, goal_bias=goal_bias)
        
        # Plan path
        path = rrt.plan()
        
        if path is None:
            print("No path found!")
            rrt.cleanup_visualization()
            return None
        
        return path, rrt
        
    except Exception as e:
        print(f"Planning error: {e}")
        return None

# RRT_star

In [146]:
from typing import List, Tuple, Optional

class RRT_STAR_Planner(RRTPlanner):
    def __init__(self, sim, start_config, goal_config, epsilon=0.2, search_radius=0.5):
        super().__init__(sim, start_config, goal_config, epsilon)
        self.search_radius = search_radius  # Radius for searching nearby nodes
        print(f"Initialized RRT* Planner with search radius {search_radius}")
        
    def get_node_cost(self, node: Node) -> float:
        """Calculate cost from start to this node"""
        cost = 0
        current = node
        while current.parent is not None:
            cost += np.linalg.norm(current.config - current.parent.config) 
            current = current.parent
        return cost

    def find_near_nodes(self, new_node: Node) -> List[Node]:
        """Find nodes within search radius of new_node"""
        near_nodes = []
        for node in self.nodes:
            if np.linalg.norm(node.config - new_node.config) < self.search_radius:
                near_nodes.append(node) 
        return near_nodes

    def get_new_cost(self, from_node: Node, to_node: Node) -> float:
        """Calculate potential cost through new path"""
        return self.get_node_cost(from_node) + np.linalg.norm(from_node.config - to_node.config)

    def extend(self) -> Optional[Node]:
        """Extend tree by one node with RRT* optimizations"""
        try:
            # Sample configuration (same as RRT)
            if np.random.random() < 0.1:
                q_rand = self.goal_config
            else:
                q_rand = self.sample_config()
            
            # Find nearest node (same as RRT)
            q_near = self.find_nearest_node(q_rand)
            
            # Generate new configuration (same as RRT)
            q_new_config = self.new_config(q_near, q_rand)
            
            # Collision check (same as RRT)
            if check_collision(self.sim, q_new_config.tolist(), self.sim.getObject("/Obstacle")):
                return None
                
            # Create new node
            q_new = Node(q_new_config)
            
            # Find nearby nodes for rewiring
            near_nodes = self.find_near_nodes(q_new)
            
            # Connect to best parent
            min_cost = float('inf')
            best_parent = None
            
            for near_node in near_nodes:
                # Calculate potential cost through this node
                potential_cost = self.get_new_cost(near_node, q_new)
                
                if potential_cost < min_cost:
                    # Check if connection is collision-free
                    path_configs = self.interpolate_path(near_node.config, q_new.config)
                    is_valid = True
                    for config in path_configs:
                        if check_collision(self.sim, config.tolist(), self.sim.getObject("/Obstacle")):
                            is_valid = False
                            break
                    
                    if is_valid:
                        min_cost = potential_cost
                        best_parent = near_node
            
            # If no valid parent found, use nearest node
            if best_parent is None:
                best_parent = q_near
            
            # Connect new node to best parent
            q_new.parent = best_parent
            best_parent.children.append(q_new)
            self.nodes.append(q_new)
            
            # Rewire tree
            for near_node in near_nodes:
                if near_node != best_parent:
                    # Calculate cost through new node
                    potential_cost = self.get_node_cost(q_new) + np.linalg.norm(q_new.config - near_node.config)
                    
                    if potential_cost < self.get_node_cost(near_node):
                        # Check if rewiring is collision-free
                        path_configs = self.interpolate_path(q_new.config, near_node.config)
                        is_valid = True
                        for config in path_configs:
                            if check_collision(self.sim, config.tolist(), self.sim.getObject("/Obstacle")):
                                is_valid = False
                                break
                        
                        if is_valid:
                            # Remove from old parent
                            if near_node.parent is not None:
                                near_node.parent.children.remove(near_node)
                            
                            # Add to new parent
                            near_node.parent = q_new
                            q_new.children.append(near_node)
            
            # Visualize new node and connections
            self._visualize_node(q_new)
            
            return q_new
            
        except Exception as e:
            print(f"RRT* extension error: {e}")
            return None

    def interpolate_path(self, start_config: np.ndarray, end_config: np.ndarray, steps: int = 10) -> List[np.ndarray]:
        """Interpolate path between configurations for collision checking"""
        path = []
        for i in range(steps):
            t = i / (steps - 1)
            config = start_config + t * (end_config - start_config)
            path.append(config)
        return path

def plan_with_rrt_star(sim, start_config, goal_config, search_radius=0.5):
    """Plan path using RRT*"""
    try:
        # Initialize RRT* planner
        rrt = RRT_STAR_Planner(sim, start_config, goal_config, epsilon=0.1, search_radius=search_radius)
        
        # Plan path
        path = rrt.plan()
        
        if path is None:
            print("No path found!")
            rrt.cleanup_visualization()
            return None
        
        return path, rrt
        
    except Exception as e:
        print(f"RRT* planning error: {e}")
        return None

# Main

In [142]:
def main_loop(sim, joints, points, n_steps=400):
    """Main execution loop with RRT path following"""
    sim.setStepping(True)
    sim.startSimulation()
    
    # Initial setup
    js = read_joints(joints)
    ps = read_points(points)
    target_thetas = solve(js, ps)
    start_config = [0, 0, 0, 0, 0]
    
    # Plan path using RRT and get visualization handles
    #result = plan_with_rrt(sim, start_config, target_thetas)
    result = plan_with_rrt_goal(sim, start_config, target_thetas, goal_bias=0.05)
    #result = plan_with_rrt_star(sim, start_config, target_thetas, search_radius=0.5)
    if result is None:
        print("Failed to find a path. Stopping execution.")
        sim.stopSimulation()
        return
        
    path_configs, rrt_planner = result  # Get both path and RRT planner object
    
    # Set up for execution
    place_thetas = [np.pi, -np.pi / 6, -np.pi / 2.7, -np.pi / 3, 0]
    base_thetas = [0, 0, 0, 0, 0]
    stage = 0
    gripper_positions = collections.deque(maxlen=100)
    current_path_index = 0
    
    # Main execution loop
    for i in range(n_steps):
        if stage == 0:  # Follow RRT path to target
            if current_path_index < len(path_configs):
                current_config = path_configs[current_path_index]
                diff_sum = trace_arm(joints, current_config)
                
                if abs(diff_sum) < 0.005:
                    current_path_index += 1
                    if current_path_index >= len(path_configs):
                        stage = 1
            
        elif stage == 1:  # grip target
            position = control_gripper(joints[-1], True)
            gripper_positions.append(position)
            if len(gripper_positions) > 5 and np.abs(gripper_positions[-1] - gripper_positions[-5]) < 0.001:
                stage = 2
                
        elif stage == 2:  # pick the target
            target_thetas[0] = np.pi
            target_thetas[1] = -np.pi / 12
            target_thetas[2] = -np.pi / 6
            diff_sum = trace_arm(joints, target_thetas)
            if abs(diff_sum) < 0.005:
                stage = 3
                
        elif stage == 3:  # move to place
            diff_sum = trace_arm(joints, place_thetas)
            if abs(diff_sum) < 0.005:
                stage = 4
                
        elif stage == 4:  # place the target
            position = control_gripper(joints[-1], False)
            gripper_positions.append(position)
            if len(gripper_positions) > 5 and np.abs(gripper_positions[-1] - gripper_positions[-5]) < 0.001:
                stage = 5
                
        elif stage == 5:  # move to base
            diff_sum = trace_arm(joints, base_thetas)
            if abs(diff_sum) < 0.005:
                stage = 6
                
        sim.step()
    
    # Clean up visualization using RRT planner's cleanup method
    rrt_planner.cleanup_visualization()
        
    sim.stopSimulation()

In [135]:
main_loop(sim, joints, points, n_steps=400)

C:\Users\joy4m\AppData\Local\Temp\ipykernel_154252\3481707928.py:6: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last five Jacobian evaluations.
  target_thetas = fsolve(


Planning RRT path...
Goal reached after 729 iterations!


# Experiment

In [ ]:
def run_experiments(sim, joints, points, n_trials=10, planner_type='rrt', **planner_params):
    """Run multiple trials and collect iteration counts
    
    Args:
        sim: Simulation instance
        joints: Robot joints
        points: Robot points
        n_trials: Number of trials to run
        planner_type: 'rrt', 'rrt_goal', or 'rrt_star'
        **planner_params: Additional parameters for specific planners
    
    Returns:
        List of iteration counts (None if path not found)
    """
    results = []
    
    for trial in range(n_trials):
        print(f"\nTrial {trial + 1}/{n_trials}")
        
        # Initialize robot position
        for i in range(len(joints)):
            sim.setJointPosition(joints[i], 0)
            
        class IterationCounter:
            def __init__(self):
                self.count = None
                
            def update(self, count):
                self.count = count
                
        counter = IterationCounter()
        
        # Override print function temporarily to capture iteration count
        original_print = print
        def custom_print(msg):
            if "Goal reached after" in msg:
                iterations = int(msg.split("after")[1].split("iterations")[0])
                counter.update(iterations)
            original_print(msg)
            
        import builtins
        builtins.print = custom_print
        
        try:
            # Run the main loop
            main_loop(sim, joints, points)
            results.append(counter.count)
            
        except Exception as e:
            print(f"Error in trial {trial + 1}: {e}")
            results.append(None)
            
        finally:
            # Restore original print function
            builtins.print = original_print
            
        # Small delay between trials
        import time
        time.sleep(1)
        
    # Print summary
    successful_trials = [r for r in results if r is not None]
    if successful_trials:
        print("\nResults Summary:")
        print(f"Success rate: {len(successful_trials)}/{n_trials}")
        print(f"Average iterations when successful: {sum(successful_trials)/len(successful_trials):.1f}")
        print(f"Min iterations: {min(successful_trials)}")
        print(f"Max iterations: {max(successful_trials)}")
    else:
        print("\nNo successful trials")
        
    print("\nAll results:", results)
    return results

# Run experiments for different planners
print("\nRunning RRT experiments...")
rrt_results = run_experiments(sim, joints, points, n_trials=10)

#print("\nRunning RRT* experiments...")
#rrt_star_results = run_experiments(sim, joints, points, n_trials=10, planner_type='rrt_star', search_radius=0.5)

#print("\nRunning RRT-Goal experiments...")
#rrt_goal_results = run_experiments(sim, joints, points, n_trials=10, planner_type='rrt_goal', goal_bias=0.2)


Running RRT-Goal experiments...

Trial 1/10


C:\Users\joy4m\AppData\Local\Temp\ipykernel_154252\3481707928.py:6: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last five Jacobian evaluations.
  target_thetas = fsolve(


Initialized RRT Goal-Biased Planner with 5.0% goal bias
Planning RRT path...


KeyboardInterrupt: 